In [1]:

##This notebook is built around using tensorflow as the backend for keras 
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Using TensorFlow backend.


In [43]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input




In [44]:
##Dimensions of our images.# dimen 
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

In [45]:
##Used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

##Automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary',
        classes = ['dogs', 'cats'])

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary',
        classes = ['dogs', 'cats'])

Found 2048 images belonging to 2 classes.
Found 834 images belonging to 2 classes.


In [46]:
##Build model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [47]:
##Compile Model
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

# keras.losses.binary_crossentropy
model.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [48]:
##Train Model
nb_epoch = 15
nb_train_samples = 500
nb_validation_samples = 832

In [49]:
model.fit_generator(train_generator, 
                    steps_per_epoch=nb_train_samples, 
                    epochs=nb_epoch,
                    validation_data=validation_generator, 
                    validation_steps=nb_validation_samples, 
                    use_multiprocessing=True, 
                    shuffle=True)



Epoch 1/15
500/500 [==============================] - 457s 915ms/step - loss: 0.6423 - acc: 0.6281 - val_loss: 0.5977 - val_acc: 0.6760
Epoch 2/15
500/500 [==============================] - 472s 945ms/step - loss: 0.4615 - acc: 0.7870 - val_loss: 0.6128 - val_acc: 0.7272
Epoch 3/15
500/500 [==============================] - 435s 869ms/step - loss: 0.2957 - acc: 0.8754 - val_loss: 0.7766 - val_acc: 0.7659
Epoch 4/15
500/500 [==============================] - 421s 842ms/step - loss: 0.1599 - acc: 0.9354 - val_loss: 1.0324 - val_acc: 0.7037
Epoch 5/15
500/500 [==============================] - 422s 844ms/step - loss: 0.1163 - acc: 0.9572 - val_loss: 1.5421 - val_acc: 0.7240
Epoch 6/15
500/500 [==============================] - 421s 843ms/step - loss: 0.0870 - acc: 0.9685 - val_loss: 1.4404 - val_acc: 0.7257
Epoch 7/15
500/500 [==============================] - 421s 843ms/step - loss: 0.0880 - acc: 0.9719 - val_loss: 1.9460 - val_acc: 0.7299
Epoch 8/15
500/500 [============================

In [50]:
##Evaluating on validation set
model.evaluate_generator(validation_generator, nb_validation_samples)

[2.156786685559582, 0.729046804540507]

In [51]:
##Data augmentation for improving the model
train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,        # normalize pixel values to [0,1]
        shear_range=0.2,       # randomly applies shearing transformation
        zoom_range=0.2,        # randomly applies shearing transformation
        horizontal_flip=True)  # randomly flip the images

# same code as before
train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 2048 images belonging to 2 classes.


In [52]:
model.fit_generator(train_generator_augmented, 
                    steps_per_epoch=nb_train_samples, 
                    epochs=15,
                    validation_data=validation_generator, 
                    validation_steps=nb_validation_samples, 
                    use_multiprocessing=True, 
                    shuffle=True)

Epoch 1/15
500/500 [==============================] - 575s 1s/step - loss: 0.7135 - acc: 0.4992 - val_loss: 0.6932 - val_acc: 0.5002
Epoch 2/15
500/500 [==============================] - 580s 1s/step - loss: 0.6932 - acc: 0.4979 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/15
500/500 [==============================] - 606s 1s/step - loss: 0.6932 - acc: 0.4887 - val_loss: 0.6931 - val_acc: 0.5003
Epoch 4/15
500/500 [==============================] - 569s 1s/step - loss: 0.6932 - acc: 0.4977 - val_loss: 0.6932 - val_acc: 0.5002
Epoch 5/15
500/500 [==============================] - 566s 1s/step - loss: 0.6932 - acc: 0.4869 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 6/15
500/500 [==============================] - 567s 1s/step - loss: 0.6932 - acc: 0.4914 - val_loss: 0.6931 - val_acc: 0.5002
Epoch 7/15
500/500 [==============================] - 783s 2s/step - loss: 0.6932 - acc: 0.4901 - val_loss: 0.6931 - val_acc: 0.5006
Epoch 8/15
500/500 [==============================] - 869s 2s/step - 

In [53]:
##Evalutating on Validation Set
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.6931472426352727, 0.5001945979606134]

In [60]:
##Test model

img=image.load_img('test/img001.jpg', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
result = model.predict(x, batch_size=1)
print result


[[0.5004587]]
